In [30]:
from gensim.models import KeyedVectors
from sklearn.feature_extraction.text import CountVectorizer
from nltk.sentiment import util

#import nltk
import numpy as np
import math
import io

In [5]:
# https://github.com/nguyenkh/AntSynDistinction
WORDVEC_PATH = r'~/adithya/Adithya/Senior/15388/Final15388/wiki_en_dLCE_100d_minFreq_100.bin'

model = KeyedVectors.load_word2vec_format(WORDVEC_PATH, binary=True)

In [13]:
model.most_similar(positive=['woman', 'king'], negative=['man'], topn=5)
model.similarity('sincere', 'skeptical')

0.4146328343605143

In [18]:
test = 'Wow. This is definitely not the usual North Korea bluffing. Consider me cautiously optimistic.'

def neg_tokenize(text):
    tokens = util.mark_negation(nltk.word_tokenize(text))
    return tokens

In [64]:
positive_keywords = ['tranquil', 'responsible', 'safe']
model.most_similar(positive=positive_keywords, topn=200)

[('justified', 0.9180777072906494),
 ('entire', 0.9110749363899231),
 ('suitable', 0.9102784395217896),
 ('beneficial', 0.9090153574943542),
 ('agreeable', 0.9057101607322693),
 ('dependable', 0.9048237204551697),
 ('even', 0.9033408164978027),
 ('snug', 0.9027947187423706),
 ('fit', 0.9026384353637695),
 ('righteous', 0.9014285206794739),
 ('square', 0.9000667333602905),
 ('layer', 0.8999127149581909),
 ('useful', 0.8974153399467468),
 ('harmless', 0.8960446119308472),
 ('adequate', 0.894891083240509),
 ('acceptable', 0.8947787284851074),
 ('exact', 0.8946834206581116),
 ('due', 0.8930257558822632),
 ('meet', 0.8928929567337036),
 ('competent', 0.8923766613006592),
 ('faithful', 0.8921601176261902),
 ('self', 0.8919709324836731),
 ('satisfactory', 0.8904110789299011),
 ('equitable', 0.890221893787384),
 ('religious', 0.8900294303894043),
 ('decent', 0.8896532654762268),
 ('equal', 0.8893912434577942),
 ('precise', 0.8891459107398987),
 ('orderly', 0.8875381350517273),
 ('fitting', 0.8

In [65]:
negative_keywords = ['scared', 'anxious', 'cautious', 'stressful', 'alert', 'emotion', 'attitude']
negative_keywords = ['scary', 'irresponsible', 'dangerous']
model.most_similar(positive=negative_keywords, topn=200)

[('insidious', 0.8102236986160278),
 ('dodgy', 0.8051842451095581),
 ('surreal', 0.795381486415863),
 ('phony', 0.7907932996749878),
 ('fantastical', 0.7826820015907288),
 ('ambivalent', 0.7812489867210388),
 ('fabulous', 0.7811815738677979),
 ('crackpot', 0.7773433923721313),
 ('unrealistic', 0.7767112255096436),
 ('mischievous', 0.7761960625648499),
 ('messy', 0.7756713032722473),
 ('half-baked', 0.7749350666999817),
 ('subtle', 0.7741920948028564),
 ('quirky', 0.7739294767379761),
 ('temperamental', 0.7729725241661072),
 ('fictional', 0.7704792022705078),
 ('contrived', 0.766952395439148),
 ('visionary', 0.7658067941665649),
 ('treacherous', 0.7649334073066711),
 ('inconsistent', 0.7626596689224243),
 ('devious', 0.7606952786445618),
 ('likable,', 0.7597777247428894),
 ('diabolical', 0.7572430372238159),
 ('perilous', 0.7537726759910583),
 ('misguided', 0.7534028887748718),
 ('hypocrite', 0.7524522542953491),
 ('obsessive,', 0.7518420815467834),
 ('funny', 0.7517601251602173),
 ('un

In [66]:
def read_stopwords(filename):
    stopwords = []
    with io.open(filename, 'r', encoding='utf-8') as f:
        for line in f.readlines():
            line = line.strip()
            stopwords.append(line)
    return frozenset(stopwords)
        

def generate_features(pos, neg, stopwords):
    pos_200 = model.most_similar(positive=positive_keywords, topn=300)
    neg_200 = model.most_similar(positive=negative_keywords, topn=300)

    uni_features = {}
    for p in pos:
        uni_features[p.lower()] = 1
    for n in neg:
        uni_features[n.lower()] = -1
    for word, sim in pos_200:
        word = word.lower()
        score = uni_features.get(word, 0)
        uni_features[word] = score + sim
    for word, sim in neg_200:
        word = word.lower()
        score = uni_features.get(word, 0)
        uni_features[word] = score - sim

    for stopword in stopwords:
        uni_features.pop(stopword, None)
    return uni_features

stopwords = read_stopwords('stopwords.txt')
uni_features = generate_features(positive_keywords, negative_keywords, stopwords)
print('pos', len([v for k, v in uni_features.items() if v > 0]))
print('neg', len([v for k, v in uni_features.items() if v < 0]))

pos 279
neg 303


In [67]:
class SentAnalyzer():
    
    def __init__(self, features):
        neg_features = self.duplicate_negation(features)
        neg_features.update(features)
        self.features = neg_features
        idxs = {feat: i for i, feat in enumerate(self.features)}
        self.idxs = idxs
        self.vectorizer = CountVectorizer(vocabulary=idxs, tokenizer=neg_tokenize)
        flipped_idxs = {i: self.features[feat] for feat, i in idxs.items()}
        self.weights = np.array([flipped_idxs[i] for i in range(len(flipped_idxs))])
    
    def duplicate_negation(self, features):
        neg_features = {
            '{}_NEG'.format(feat): -1 * score for feat, score in features.items()
        }
        return neg_features
    
    def normalize(self, score, alpha=5):
        """Normalizing function as described by VADER Sentiment analyzer"""
        return score / np.sqrt((score * score) + alpha)
    
    def polarize(self, passages):
        """Calculate the polarity of a passage.
        
        passage [str]: a list of strings representing tokens
        """
        bow_vec = self.vectorizer.transform(passages)
        return bow_vec.dot(self.weights)
    
    def predict(self, passage):
        return self.normalize(self.polarize(passage))
    
SA = SentAnalyzer(uni_features)
test2 = "I'm still really skeptical myself. This is all just too good to be true. \nWhat the fuck is Kim actually planning? How the fuck is North Korea going to unify with South Korea when NK has the most fucked up and brainwashed population in existence?"
test3 = "I'm really optimistic. I hope this succeeds."
test4 = "Is it really happening? There is no obvious reason to believe him and this is all happening so fast? Could something amazing actually be happening here?"
test5 = "Got the warning on my phone. I was just talking to friends on discord then heard the weird note from my phone that I hear for the first time. I checked it says missile. I was terrified checking every news source and calling friends. Then got another saying it passed over. That was terrifying."
test6 = "I've been living in northern Japan for some time and have never seen this kind of response to a missile launch before. The alerts, the news broadcasts, it was not taken lightly. Regardless of where the missile ended up, I think the message it sent is one that will have some lasting implications on how we perceive North Korea's aggression. It's only a matter of time before they end their war with Poseidon and make the mistake of targeting actual human lives. Stay safe out there, everyone."
test7 = "Their targeting systems and missiles are far from foolproof. A missile failure could have dropped this missile on an inhabited area in Japan. That would likely have provoked a full scale war. The level of irresponsibility here is just staggering."
test8 = "Man, that must be scary as shit. The other day I just arrived in Brussels and sat down in the Grand Place (a big open square, right in the center of Brussels) and then got a BBC alert 'Machete-wielding man attacks soldiers in central Brussels'. My heart skipped a beat. It was just a couple of streets away. I felt kinda sick. I did hear police sirens earlier but didn't think much of it. There were also soldiers and police all over the place."
print(SA.predict([test, test2, test3, test4, test5, test6, test7, test8]))

[0.         0.61447401 0.         0.         0.04363161 0.76535403
 0.35263653 0.72628443]
